<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_error
from xgboost import XGBRegressor

In [ ]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 17.3MB/s]


In [ ]:
data = pd.read_csv('ElectricCarData.csv')

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [ ]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [ ]:
enc = LabelEncoder()

categorical_variables = ['Brand', 'Model', 'RapidCharge', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment']

for cat in categorical_variables:
    data[cat] = enc.fit_transform(data[cat])

In [ ]:
data.sample(100)

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
14,10,35,9.7,165,250,153,210,1,1,2,2,2,5,34459
6,31,83,9.6,150,190,168,220,1,1,2,1,2,5,31900
90,1,93,4.5,210,320,270,510,1,0,2,6,4,5,93800
89,27,24,9.0,160,320,181,440,1,2,2,6,2,5,37500
39,18,20,5.0,200,350,171,440,1,0,2,6,2,5,45000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,8,59,6.6,180,360,194,380,1,2,2,6,3,5,46900
69,8,56,6.0,180,430,209,410,1,0,2,6,3,5,62900
99,1,94,4.5,210,335,258,540,1,0,2,6,4,5,96050
60,25,75,11.4,135,255,161,230,1,1,2,1,1,5,29234


In [ ]:
data.isin(['-']).sum()

,0
Brand,0
Model,0
AccelSec,0
TopSpeed_KmH,0
Range_Km,0
Efficiency_WhKm,0
FastCharge_KmH,5
RapidCharge,0
PowerTrain,0
PlugType,0


In [ ]:
data['FastCharge_KmH'] = data['FastCharge_KmH'].replace('-', np.nan).astype(float)

In [ ]:
data.isnull().sum()

,0
Brand,0
Model,0
AccelSec,0
TopSpeed_KmH,0
Range_Km,0
Efficiency_WhKm,0
FastCharge_KmH,5
RapidCharge,0
PowerTrain,0
PlugType,0


In [ ]:
data['FastCharge_KmH'] = data['FastCharge_KmH'].fillna(data['FastCharge_KmH'].median())

## Split the dataset for training and testing in ratio 80:20

In [ ]:
X = data.drop('PriceEuro', axis=1)
Y = data['PriceEuro']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

In [ ]:
model1 = DecisionTreeRegressor(max_depth=5, min_samples_split=7, max_features=2)

model1.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, max_features=2, min_samples_split=7)

## Predict the outcomes for X test

In [ ]:
y_pred = model1.predict(x_test)

In [ ]:
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))

Mean Absolute Error: 8860.496031746032
Mean Squared Error: 226777475.4408597
R2 Score: 0.6559912112969373


## Assess the model performance, by using sklearn metrics for regression

## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [ ]:
model2 = XGBRegressor(n_estimators=35, max_depth=3, learning_rate=0.16, objective="reg:squarederror")
model2.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.16, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=35, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [ ]:
y_pred2 = model2.predict(x_test)

## Assess the model performance, by using sklearn metrics for regression

In [ ]:
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred2))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred2))
print("R2 Score:", r2_score(y_test, y_pred2))

Mean Absolute Error: 6472.123139880952
Mean Squared Error: 122399151.05399176
R2 Score: 0.8143272995948792


## Compare the performances of both model for at least three regression metircs

In [ ]:
print("Mean Absolute Error DecisionTree:", mean_absolute_error(y_test, y_pred))
print("Mean Absolute Error XGBoost:", mean_absolute_error(y_test, y_pred2))

Mean Absolute Error DecisionTree: 8860.496031746032
Mean Absolute Error XGBoost: 6472.123139880952


In [ ]:
print("Mean Squared Error DecisionTree:", mean_squared_error(y_test, y_pred))
print("Mean Squared Error XGBoost:", mean_squared_error(y_test, y_pred2))

Mean Squared Error DecisionTree: 226777475.4408597
Mean Squared Error XGBoost: 122399151.05399176


In [ ]:
print("R2 Score DecisionTree:", r2_score(y_test, y_pred))
print("R2 Score XGBoost:", r2_score(y_test, y_pred2))

R2 Score DecisionTree: 0.6559912112969373
R2 Score XGBoost: 0.8143272995948792
